## COMPAS プログラム説明書サンプル

### プログラム名

* SOPOS (太陽位置)

### 内容説明

* 緯度、経度、⽉、⽇、時刻をもとに太陽位置を求めるプログラムである。
* 太陽位置を表すものとして、 sinh、sinA、cosA（h：太陽⾼度、A：太陽⽅位⾓）の3つを求める。

### 引数
  
* FN 北緯（南緯はマイナス） (°) 
* EL 東経（⻄経はマイナス） (°) 
* M ⽉ 
* N ⽇ 
* HJ ⽇本標準時（時） 

### プログラム（python）

In [139]:
import math

1. 通し日数の算出

In [140]:
def D(M,N):
    """
    1⽉1⽇を1として順に数えた通し⽇数
    """

    y = 30*(M-1)+(M+M/8)/2-(M+7)/10+N

    return y

2. 日赤緯の算出

⽇⾚緯 δ [°] は次式で算出される（滝沢による）。 

$$
\sigma = 0.3622133 - 23.24763*cos(\omega + 0.1532310) ― 0.3368908 * cos (2\omega + 0.2070988) - 0.1852646 * cos(3\omega + 0.6201293) 
$$

ここで、$ \omega = 2 \pi d/366 $  であり、dは1⽉1⽇を1として順に数えた通し日数である。

In [141]:
def DEL(W):
    """
    日赤緯[°]を求める関数（滝沢による）
    """

    y = 0.3622133 \
        - 23.24763  * math.cos( W + 0.153231) \
        - 0.3368908 * math.cos(2.0 * W + 0.2070988) \
        - 0.1852646 * math.cos(3.0 * W + 0.6201293)

    return y

3. 均時差の算出

均時差 e [h] は次式で算出される（滝沢による）。

$$
e = -0.0002786409 + 0.1227715 * cos(\omega + 1.498311) - 0.1654575 *cos(2\omega - 1.261546) - 0.005353830 * cos(3\omega - 1.157100)
$$

ここで、$ \omega = 2 \pi d/366 $  であり、dは1⽉1⽇を1として順に数えた通し日数である。

In [142]:
def EQT(W):

    y = -0.0002786409 \
        + 0.1227715   * math.cos( W + 1.498311) \
        - 0.1654575   * math.cos(2.0 * W - 1.261546) \
        - 0.005353830 * math.cos(3.0 * W - 1.157100)

    return y

4. 太陽位置の計算

時角 t は次式で算出される。

$$
t = 15 * (t_s - 12 + e) + L + L_0
$$

ここで、  
    $t_s$：中央標準時 [時]  
    $𝐿$、$𝐿_0$ ：その⼟地と中央標準時の基準となる⼟地の経度 [°]  

太陽高度をh、太陽方位角をAとすると、太陽位置を表すsinh、sinA、cosAは次式で算出される。


$$
sinh = sin \varphi \cdot sin \delta + cos \varphi \cdot cos \delta \cdot cos t 
$$

$$
cosh = \sqrt{1-sinh^{2}}
$$

$$
sinA = \frac{sin t \cdot cos \delta}{cosh}
$$

$$
cosA = \frac{sinh \cdot sin \varphi - sin \delta}{cosh \cdot cos \varphi}
$$

ここで、
$\varphi$：その⼟地の緯度

ただし、sinh<0、sinA<0、cosA<0 などのときは、SH=0、SA=0、CA=0 とする。

In [143]:
def SOPOS(FN,EL,M,N,HJ):
    """
    太陽位置を求める関数。
    HJ には真太陽時を⼊れる。
    # 出力
    # SH : sinh 
    # SA : sinA 
    # CA : cosA 
    """
    
    PAI  = 3.141593
    RADI = 0.01745329
    ELS  = 135.0

    W = PAI * D(M,N) / 183.0

    # 時⾓ T [°]
    T = 15.0 * (HJ - 12.0 + EQT(W)) + EL - ELS
    # T = 15.0 * (HJ - 12.0)

    # TJ 太陽時（時）
    TJ = T/15.+12

    FN1=FN*RADI
    SF=math.sin(FN1)
    CF=math.cos(FN1)
    DEL1=DEL(W)*RADI

    SD=math.sin(DEL1)
    CD=math.cos(DEL1)

    T1=T*RADI
    CT=math.cos(T1)
    ST=math.sin(T1)

    SH=SF*SD+CF*CD*CT

    if (SH > 0):
        CH=math.sqrt(1.0-SH**2)
        SA=CD*ST/CH
        CA=(SH*SF-SD)/(CH*CF)
    else:
        SH=0.0
        SA=0.0
        CA=0.0

    return SH, SA, CA, TJ


### 例題

In [144]:
FN = 35.68     # 北緯
EL = 139.77    # 東経
M  = 8         # 月
N  = 23        # 日

for hour in range(0,24):

    # 日本標準時
    HJ = hour+1

    SH, SA, CA, TJ = SOPOS(FN,EL,M,N,HJ)

    print(HJ, SH, SA, CA, TJ)


1 0.0 0.0 0.0 1.2685730341959562
2 0.0 0.0 0.0 2.2685730341959562
3 0.0 0.0 0.0 3.2685730341959562
4 0.0 0.0 0.0 4.268573034195955
5 0.0 0.0 0.0 5.2685730341959545
6 0.17349096726437302 -0.9920883576623617 -0.1255415890882299 6.2685730341959545
7 0.3769955613099495 -0.9997052272849931 0.024278767247543513 7.2685730341959545
8 0.5628225104171131 -0.9820478002305052 0.18863222964919293 8.268573034195954
9 0.7183080187337056 -0.923090124611318 0.38458369940113357 9.268573034195954
10 0.8328560088621756 -0.7749543462527956 0.6320172159236663 10.268573034195954
11 0.8986602268160498 -0.4249688324638148 0.9052079824185943 11.268573034195954
12 0.9112362251759273 0.1670653782861992 0.9859458196970513 12.268573034195954
13 0.8697269706824675 0.6470575990799504 0.7624411213155351 13.268573034195954
14 0.7769612496199321 0.8706469581663843 0.4919084002490942 14.268573034195954
15 0.6392608907610501 0.9616918707623388 0.2741327155040668 15.268573034195954
16 0.4660099433071685 0.9952202806361371 

### 制約・⽋点

* 時刻 HJ は、⽇本標準時となっている。ELS=135.0 は明⽯の東経を⽰しているので、外国の時刻のときに は注意を要す。
* ⽇出、⽇没を求めるプログラムにはなっていない。
* ⽇⾚緯、均時差などは、ある⼀⽇では同⼀の値をとるので、時刻別に幾度もCALLするときなどは、DEL(W)、 EQT(W)の計算は無駄になっている。
* W の計算において、1 年を 366 ⽇とみなしているので、最⼤ 1 ⽇分の太陽位置の誤差が出る。
* 現状では、中央標準時での太陽位置を求めることになっているが、太陽時における太陽位置を求めようにするときには、コメント⾏を⽣かすとよい。
